### Create SQL DB

In [4]:
import sqlite3

def create_database(db_file):
    """Creates an SQLite database file."""
    conn = sqlite3.connect(db_file)
    conn.close()

# Example usage:
db_file = 'data/sqldb'

# Create the database file
create_database(db_file)

In [5]:
import os

db_file = 'data/sqldb'

if os.path.exists(db_file):
    print(f"Database file '{db_file}' exists.")
else:
    print(f"Database file '{db_file}' does not exist.")

Database file 'data/sqldb' exists.


### CSV to SQL table

In [6]:
import pandas as pd
from sqlalchemy import create_engine

def csv_to_sqlite(csv_directory, db_file):
    """
    Converts CSV files in a directory to an SQLite database.
    Args:
        csv_directory (str): Path to the directory containing CSV files.
        db_file (str): Path to the output SQLite database file.
    """

    engine = create_engine(f'sqlite:///{db_file}')

    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(csv_directory, filename)
            df = pd.read_csv(file_path)
            
            # Identify date columns using pandas' infer_datetime_format
            date_cols = df.select_dtypes(include=['datetime', 'datetime64', 'object']).columns

            # Convert identified date columns to datetime objects
            for col in date_cols:
                try:
                    df[col] = pd.to_datetime(df[col])
                except ValueError:
                    # Handle potential conversion errors, e.g., log or skip
                    print(f"Failed to convert {col} to datetime")
                
            table_name = filename.split('.')[0]  # Extract table name from filename
            df.to_sql(table_name, engine, index=False, if_exists='replace')

# Example usage:
csv_directory = 'data/csv'
db_file = 'data/sqldb'

csv_to_sqlite(csv_directory, db_file)

C:\Users\Sankar\AppData\Local\Temp\ipykernel_19304\250803852.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
C:\Users\Sankar\AppData\Local\Temp\ipykernel_19304\250803852.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


Failed to convert customer_name to datetime
Failed to convert city to datetime


C:\Users\Sankar\AppData\Local\Temp\ipykernel_19304\250803852.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
C:\Users\Sankar\AppData\Local\Temp\ipykernel_19304\250803852.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


Failed to convert partner_name to datetime
Failed to convert partner_code to datetime


C:\Users\Sankar\AppData\Local\Temp\ipykernel_19304\250803852.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


Failed to convert tran_status to datetime


### Select the created tables and display Results

In [7]:
def list_tables(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_names = cursor.fetchall()

    for table_name in table_names:
        print(table_name[0])

    conn.close()

# Example usage:
db_file = 'data/sqldb'

# List tables
list_tables(db_file)

customer_data
partner_data
transaction_data


In [8]:
def query_table(db_file, table_name):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    for row in rows:
        print(row)

    conn.close()

# Example usage:
db_file = 'data/sqldb'

# Query a specific table
table_name = 'customer_data'  # Replace with the actual table name
query_table(db_file, table_name)

(10001, 'Sankar', 'New York')
(10002, 'Ragul', 'Los Angeles')
(10003, 'Aswin', 'Boston')
(10004, 'Sriram', 'Seattle')
(10005, 'Rinika', 'Chicago')
(10006, 'Ponvignesh', 'Los Angeles')
(10007, 'Ramakrishnan', 'New York')
(10008, 'Yogan', 'Los Angeles')
(10009, 'Subrat', 'Chicago')
(10010, 'Suriya', 'New York')
(10011, 'Ishwarya', 'Los Angeles')
(10012, 'Swetha', 'Boston')
(10013, 'Kavya', 'Seattle')
(10014, 'Ken', 'New York')


### Get Schema

In [9]:
import sqlite3

def print_table_schema(db_file):
    """Prints the schema of all tables in an SQLite database.

    Args:
        db_file (str): Path to the SQLite database file.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get a list of all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_names = cursor.fetchall()

    for table_name in table_names:
        table_name = table_name[0]  # Extract the table name from the tuple
        print(f"\nTable: {table_name}")

        # Get the column information for the table
        cursor.execute(f"PRAGMA table_info('{table_name}')")
        columns = cursor.fetchall()

        for column in columns:
            cid, name, type_, notnull, dflt_value, pk = column
            print(f"  - {name} ({type_})")
    conn.close()

# Replace 'your_database.db' with the actual path to your database file
db_file = 'data/sqldb'
print_table_schema(db_file)


Table: customer_data
  - customer_id (BIGINT)
  - customer_name (TEXT)
  - city (TEXT)

Table: partner_data
  - partner_id (BIGINT)
  - partner_name (TEXT)
  - partner_code (TEXT)

Table: transaction_data
  - customer_id (BIGINT)
  - partner_id (BIGINT)
  - tran_amt (BIGINT)
  - tran_dt (DATETIME)
  - tran_status (TEXT)


### Delete SQL DB

In [3]:
import os
import sqlite3

def delete_database(db_file):
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        os.remove(db_file)
        print(f"Database file '{db_file}' deleted successfully.")
    except Exception as e:
        print(f"Error deleting database: {e}")

# Example usage:
db_file = 'data/sqldb'
delete_database(db_file)

Database file 'data/sqldb' deleted successfully.
